<a href="https://colab.research.google.com/github/Gabriel-MIAUNI/roboflownotebooks/blob/main/2_1_TicTacToe_minimax.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*   Autor: Clederson Cruz
*   Año: 2017
*   Licencia: GNU GENERAL PUBLIC LICENSE (GPL)
*   [texto do link](https://github.com/Cledersonbc/tic-tac-toe-minimax)



# **Implementación de MiniMax** #


In [ ]:
from math import inf as infinity
from random import choice
import platform
import time
from os import system

## Juego de Tres en Raya ##

### Tablero ###

In [ ]:
HUMAN = -1
COMP = +1

#Comienzo del tablero
board = [
    [0, 0, 0],
    [0, 0, 0],
    [0, 0, 0],
]

def clean():
    """
    Limpiar la pantalla
    """
    os_name = platform.system().lower()
    if 'windows' in os_name:
        system('cls')
    else:
        system('clear')


def render(state, c_choice, h_choice):
    """
    Imprimir el tablero actual
    :param state: estado actual del tablero
    """

    chars = {
        -1: h_choice,
        +1: c_choice,
        0: ' '
    }
    str_line = '---------------'

    print('\n' + str_line)
    for row in state:
        for cell in row:
            symbol = chars[cell]
            print(f'| {symbol} |', end='')
        print('\n' + str_line)

### Relgas del Juego ###

In [ ]:
def wins(state, player):
    """
    La función que permite saber si el jugador gana.
    Posibilidades de ganar:
    * 3 filas      [X X X] or [O O O]
    * 3 columnas   [X X X] or [O O O]
    * 2 diagonales [X X X] or [O O O]
    :param state: estado acutal del tablero
    :param player: computador o humano
    :return: True si el jugador gana
    """
    win_state = [
        [state[0][0], state[0][1], state[0][2]],
        [state[1][0], state[1][1], state[1][2]],
        [state[2][0], state[2][1], state[2][2]],
        [state[0][0], state[1][0], state[2][0]],
        [state[0][1], state[1][1], state[2][1]],
        [state[0][2], state[1][2], state[2][2]],
        [state[0][0], state[1][1], state[2][2]],
        [state[2][0], state[1][1], state[0][2]],
    ]
    if [player, player, player] in win_state:
        return True
    else:
        return False


def game_over(state):
    """
    Esta funcion solo se activa si gana o empate un jugador
    :param state: el estado actual del tablero
    :return: True Si un jugado gana
    """
    return wins(state, HUMAN) or wins(state, COMP)


def empty_cells(state):
    """
    Cada cela vacia se anhade en una lista de celdas
    :param state: el estado actual del tablero
    :return: lista vacia de celdas del tablero
    """
    cells = []

    for x, row in enumerate(state):
        for y, cell in enumerate(row):
            if cell == 0:
                cells.append([x, y])

    return cells


def valid_move(x, y):
    """
    Valida los movimiento en el tablero
    :param x: X cordenada
    :param y: Y condernada
    :return: True Si el board[x][y] is vacio
    """
    if [x, y] in empty_cells(board):
        return True
    else:
        return False


def set_move(x, y, player):
    """
    Conjuto de movimiento del tablero, si la coordenadas son validas
    :param x: X coordenada
    :param y: Y coordenada
    :param player: el jugador actual
    """
    if valid_move(x, y):
        board[x][y] = player
        return True
    else:
        return False


### Jugador Humano ###

In [ ]:
def human_turn(c_choice, h_choice):
    """
    El humano realiza un movimiento valido
    :param c_choice: computer's choice X or O
    :param h_choice: human's choice X or O
    :return:
    """
    depth = len(empty_cells(board))
    if depth == 0 or game_over(board):
        return

    # Dictionary of valid moves
    move = -1
    moves = {
        1: [0, 0], 2: [0, 1], 3: [0, 2],
        4: [1, 0], 5: [1, 1], 6: [1, 2],
        7: [2, 0], 8: [2, 1], 9: [2, 2],
    }

    clean()
    print(f'Turno del Humano [{h_choice}]')
    render(board, c_choice, h_choice)

    while move < 1 or move > 9:
        try:
            move = int(input('Seleccione del (1..9): '))
            coord = moves[move]
            can_move = set_move(coord[0], coord[1], HUMAN)

            if not can_move:
                print('Movimiento malo')
                move = -1
        except (EOFError, KeyboardInterrupt):
            print('Bye')
            exit()
        except (KeyError, ValueError):
            print('Mala seleccion')


## Algoritmo MiniMax ##

MiniMax es  algoritmo de IA que se aplica en juegos de dos jugadores, como la damas, ajedrez, entre otros. Estos juegos se conocen como juegos de suma cero ($\sum = 0$), porque es una representación matemática: un jugador gana ($+1$) y el otro jugador pierde ($-1$) o ambos o ninguno gana ($0$).

El algoritmo busca la mejro guada que lleve al jugador máximo a ganar o empate de forma recursiva. Por lo cual, se considera el estado actual del juego, y las jugadas disponibles en ese estado, luego, por cada jugada válida, se alterna min y max hasta encontrar un estado terminal (ganar, perder o empate).

In [ ]:
def evaluate(state):
    """
    Funcion de evaluación del estado
    :param state: el estado actual del tablero
    :return: +1 Si la computadora gana; -1 si el jugador humano gana; 0 empate
    """
    if wins(state, COMP):
        score = +1
    elif wins(state, HUMAN):
        score = -1
    else:
        score = 0

    return score

def ai_turn(c_choice, h_choice):
    """
    Esta funcion es llamda si la profundidad < 9,
    de manera contrario se escoge una coordenada aleatoria
    :param c_choice: computer's choice X or O
    :param h_choice: human's choice X or O
    :return:
    """
    depth = len(empty_cells(board))
    if depth == 0 or game_over(board):
        return

    clean()
    print(f'Computer turn [{c_choice}]')
    render(board, c_choice, h_choice)

    if depth == 9:
        x = choice([0, 1, 2])
        y = choice([0, 1, 2])
    else:
        move = minimax(board, depth, COMP)
        x, y = move[0], move[1]

    set_move(x, y, COMP)
    time.sleep(1)

def minimax(state, depth, player):
    """
    Función IA que escoge el mejor movimiento
    :param state: estado actual del tablero
    :param depth: indice del nodo en el arbol (0 <= depth <= 9),
    pero nunca 9 en este caso (funcion iaturn())
    :param player: un humano o una computadora
    :return: lista con [la mejor fila, mejor columna, mejor score]
    """
    if player == COMP:
        best = [-1, -1, -infinity]
    else:
        best = [-1, -1, +infinity]

    if depth == 0 or game_over(state):
        score = evaluate(state)
        return [-1, -1, score]

    for cell in empty_cells(state):
        x, y = cell[0], cell[1]
        state[x][y] = player
        score = minimax(state, depth - 1, -player)
        state[x][y] = 0
        score[0], score[1] = x, y

        if player == COMP:
            if score[2] > best[2]:
                best = score  # max value
        else:
            if score[2] < best[2]:
                best = score  # min value

    return best

## Empezando a jugar ##



In [ ]:
clean()
h_choice = ''  # X or O
c_choice = ''  # X or O
first = ''  # if human is the first

 # El humano escoge si ser 0 o X
while h_choice != 'O' and h_choice != 'X':
  try:
    print('')
    h_choice = input('Escoge X or O\nEscoge: ').upper()
  except (EOFError, KeyboardInterrupt):
    print('Bye')
    exit()
  except (KeyError, ValueError):
    print('Bad choice')

  # Setting computer's choice
  if h_choice == 'X':
    c_choice = 'O'
  else:
    c_choice = 'X'

# Humano comienza primero
clean()
while first != 'Y' and first != 'N':
  try:
    first = input('First to start?[y/n]: ').upper()
  except (EOFError, KeyboardInterrupt):
    print('Bye')
    exit()
  except (KeyError, ValueError):
    print('Bad choice')


# Principal bucle del Juego
while len(empty_cells(board)) > 0 and not game_over(board):
  if first == 'N':
    ai_turn(c_choice, h_choice)
    first = ''

  human_turn(c_choice, h_choice)
  ai_turn(c_choice, h_choice)


# Mensaje de Game over
if wins(board, HUMAN):
  clean()
  print(f'Turno humano [{h_choice}]')
  render(board, c_choice, h_choice)
  print('GANASTE!')
elif wins(board, COMP):
  clean()
  print(f'Turno Computadora [{c_choice}]')
  render(board, c_choice, h_choice)
  print('PERDISTE!')
else:
  clean()
  render(board, c_choice, h_choice)
  print('EMPATE!')




Escoge X or O
Escoge: X
First to start?[y/n]: y
Human turn [X]

---------------
|   ||   ||   |
---------------
|   ||   ||   |
---------------
|   ||   ||   |
---------------
Use numpad (1..9): 2
Computer turn [O]

---------------
|   || X ||   |
---------------
|   ||   ||   |
---------------
|   ||   ||   |
---------------
Human turn [X]

---------------
| O || X ||   |
---------------
|   ||   ||   |
---------------
|   ||   ||   |
---------------
Use numpad (1..9): 5
Computer turn [O]

---------------
| O || X ||   |
---------------
|   || X ||   |
---------------
|   ||   ||   |
---------------
Human turn [X]

---------------
| O || X ||   |
---------------
|   || X ||   |
---------------
|   || O ||   |
---------------
Use numpad (1..9): 3
Computer turn [O]

---------------
| O || X || X |
---------------
|   || X ||   |
---------------
|   || O ||   |
---------------
Human turn [X]

---------------
| O || X || X |
---------------
|   || X ||   |
---------------
| O || O ||   |